In [1]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils
from torchvision.transforms import Resize, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler


In [3]:
!which python

/opt/conda/bin/python


In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

1.10.2
False


In [ ]:
data_path = '/opt/ml/input/data'
train_img_dir = os.path.join(data_path, 'train/images')
test_img_dir = os.path.join(data_path, 'eval/images')
print(train_img_dir)
print(test_img_dir)

sample_submission = pd.read_csv(os.path.join(data_path, 'eval/info.csv'))
train_df = pd.read_csv(os.path.join(data_path,'train/train.csv'))
train_df_full_path = pd.read_csv('/opt/ml/workspace/data/train/train.csv')

In [ ]:
train_df = train_df.drop(['path'], axis=1)
train_df = train_df_full_path.merge(train_df, how = 'inner', on = 'id')

In [ ]:
train_df.head()

In [ ]:
train_df.to_csv('train_info.csv')

In [ ]:
print(train_df['mask'].unique())
print(train_df['gender'].unique())
print(train_df['race'].unique())

In [ ]:
clean = {'mask': {'wear' : 0, 'incorrect': 1, 'not wear': 2, },
         'gender' : {'female': 1, 'male' : 0}}

train_df['mask_encode'] = train_df['mask'].apply(lambda x : clean['mask'][x])
train_df['gender_encode'] = train_df['gender'].apply(lambda x: clean['gender'][x])

def get_age_range(age):
    if age < 30:
        return "< 30"
    elif age >= 30 and age < 60:
        return ">= 30 and < 60"
    elif age >= 60:
        return ">= 60"

train_df['age_range'] = train_df['age'].apply(lambda x : get_age_range(x))

In [ ]:
train_df.head()

In [ ]:
sample_submission.head()

In [ ]:
print("<data info>")
print(f"train data 개수: {len(train_df)}")
print("결측치 개수:")
print(pd.DataFrame(train_df.isnull().sum()))

In [ ]:
example = '/opt/ml/input/data/train/images/000001_female_Asian_45'
files = [i for i in os.listdir(example) if '._' not in i]
print(len(files))

In [ ]:
fig = plt.figure(figsize = (12,12))

for i, file in enumerate(files):
    img = Image.open(os.path.join(example, file))
    ax = fig.add_subplot(2, 4, i+1)
    ax.imshow(img)
    ax.set_title(file.split('.')[0], fontsize=15)
    ax.axis('off')
plt.tight_layout()
plt.show()


### EDA

#### 1. class imablance
    
    1) label count
    2) mask count
    3) gender count
    4) age count

In [ ]:
fig, axes = plt.subplots(4, 2, figsize = (12, 12))
axes = axes.flatten()
sns.countplot(x = 'label', data=train_df, ax = axes[0])
sns.kdeplot(x = 'label', data=train_df, fill=True, ax=axes[1])
sns.countplot(x = 'mask', data=train_df, ax = axes[2])
sns.kdeplot(x = 'mask_encode', data=train_df, fill=True, ax=axes[3])
sns.countplot(x = 'gender', data=train_df, ax = axes[4])
sns.kdeplot(x = 'gender_encode', data=train_df, fill=True, ax=axes[5])
sns.countplot(x = 'age_range', data=train_df, ax = axes[6])
sns.kdeplot(x = 'age', data=train_df, fill=True, ax=axes[7])
plt.tight_layout()
plt.show()

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, transform):
        self.x = df['full_path']
        self.y = df['label']
        self.transform = transform
    def __getitem__(self, index):
        image = Image.open(self.x.iloc[index])
        label = self.y.iloc[index]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)
    def __len__(self):
        return len(self.x)

In [ ]:
transform = transforms.Compose([
    # Resize((512, 384)),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2))
])

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train_df, test_size=0.2, shuffle=True, stratify=train_df['label'], random_state=5)
print(train.shape)
print(val.shape)

In [ ]:
BATCH_SIZE=64

In [ ]:
train_dataset = CustomDataset(train, transform)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

valid_dataset = CustomDataset(val, transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
len(train_dataset)

In [ ]:
next(iter(train_dataloader))

In [ ]:
model = torchvision.models.resnet18(pretrained=True)

In [ ]:
model

In [ ]:
import math

output_class_num = 18
model.fc = torch.nn.Linear(in_features=512, out_features=output_class_num, bias=True)

torch.nn.init.xavier_uniform_(model.fc.weight)
stdv = 1. / math.sqrt(model.fc.weight.size(1))
model.fc.bias.data.uniform_(-stdv, stdv)

model.fc.weight.shape[0]

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model.to(device)
LEARNING_RATE = 0.0001
NUM_EPOCH = 5

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
dataloaders = {
    "train": train_dataloader,
    "test": valid_dataloader
}

In [ ]:
next(iter(train_dataloader))

In [ ]:
best_test_accuracy = 0.
best_test_loss = 9999.

for epoch in range(NUM_EPOCH):
    for phase in ['train', 'test']:
        running_loss = 0.
        running_acc = 0.
        if phase == 'train':
            model.train()
        else:
            model.eval()
        
        for ind, (images, labels) in enumerate(dataloaders[phase]):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                logits = model(images)
                _, preds = torch.max(logits, 1)
                loss = loss_fn(logits, labels)
                if phase == "train":
                    loss.backward()
                    optimizer.step()
            running_loss += loss.item()*images.size(0)
            running_acc += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        print(f'현재 epoch-{epoch+1}의 {phase}-데이터 셋에서 평균 loss: {epoch_loss:.3f}, 평균 accuracy: {epoch_acc:.3f}')
        if phase == 'test' and best_test_accuracy < epoch_acc:
            best_test_accuracy = epoch_acc
        if phase == 'test' and best_test_loss > epoch_loss:
            best_test_loss = epoch_loss
print("학습 종료")
print(f'최고 accuracy: {best_test_accuracy}, loss: {best_test_loss}')

In [ ]:
test_dir = '/opt/ml/input/data/eval'
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    # Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')